In [ ]:
!pip install xlsxwriter

**KARAR AĞACI KODU**

In [4]:
# -- coding: utf-8 --
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

# ========== DOSYA YOLU ==========
PATH = "/kaggle/input/vggkjgj/2024_BAKENT_TABLO-1 SHENERGY ALIMA.xlsx"

# ========= Yardımcı: normalize =========
TR_MAP = str.maketrans({
    "İ":"i","I":"i","ı":"i","ş":"s","Ş":"s","ğ":"g","Ğ":"g",
    "ü":"u","Ü":"u","ö":"o","Ö":"o","ç":"c","Ç":"c"
})
def normalize(s: str) -> str:
    s = str(s)
    s = s.translate(TR_MAP).lower()
    s = re.sub(r"[^a-z0-9]+"," ", s).strip()
    return s

# ========= Header bulma =========
def looks_like_header(row):
    vals = [str(x) for x in row.tolist()]
    keys = ["cbs","kod","unsur","yuk","fider","trafo","neden","tip","tarih","sure","sonuc","asset","yatirim"]
    text_ratio = sum(normalize(v)!='' for v in vals)/max(1,len(vals))
    hit = any(any(k in normalize(v) for k in keys) for v in vals)
    return text_ratio>0.3 and hit

raw = pd.read_excel(PATH, header=None)
header_row = 0
for i in range(min(25, len(raw))):
    if looks_like_header(raw.iloc[i]):
        header_row = i
        break

df = pd.read_excel(PATH, header=header_row)
df.columns = df.columns.map(lambda x: str(x).strip())

# ========= Akıllı kolon bulucu =========
def find_col_by_tokens(token_sets, prefer=None):
    norm_cols = [(c, normalize(c)) for c in df.columns]
    for tokens in token_sets:
        toks = [normalize(t) for t in tokens]
        for c, nc in norm_cols:
            if all(t in nc for t in toks):
                return c
    if prefer:
        pnorm = normalize(prefer)
        for c,nc in norm_cols:
            if pnorm in nc:
                return c
    return None

def pick_code_column():
    cands = [['cbs','kod'],['asset','id'],['unsur','kod'],['kod']]
    return find_col_by_tokens(cands)

CODE_COL = pick_code_column()
if CODE_COL is None:
    raise ValueError("CBS/Asset kodu sütunu bulunamadı.")

COL_TRAF_FIDER = find_col_by_tokens([['trafo','fider'], ['trafo'], ['fider']])
COL_TIP_DETAY  = find_col_by_tokens([['sebeke','unsuru','tipi'], ['unsur','tipi'], ['3c']])
COL_YUK        = find_col_by_tokens([['yuzdelik','yuk','profil'], ['yuk','profil'], ['yuk','%']], prefer="yuk")
COL_NEDEN      = find_col_by_tokens([['nedenine','iliskin','aciklama'], ['aciklama'], ['neden']])
COL_SURE       = find_col_by_tokens([['kesinti','suresi'], ['saat']])

# ========= Sayı/Tarih dönüştürme =========
def to_float(x):
    """%92, 92%, ' 92 ', '0.92' gibi değerleri doğru parse eder."""
    try:
        s = str(x).strip().replace(",", ".")
        s = re.sub(r"[^0-9.]+", "", s)
        if s == "": return np.nan
        v = float(s)
        return v*100 if v <= 1.5 else v
    except:
        return np.nan

def parse_date_or_year(x):
    """ISO (YYYY-mm-dd[ HH:MM:SS]) için dayfirst=False; TR (dd.mm.yyyy[ HH:MM:SS]) için dayfirst=True."""
    if pd.isna(x):
        return pd.NaT
    s = str(x).strip()

    # ISO-like
    if re.match(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$", s):
        return pd.to_datetime(s, dayfirst=False, errors="coerce")

    # dd.mm.yyyy ...
    if re.match(r"^\d{1,2}\.\d{1,2}\.\d{2,4}(?:\s+\d{1,2}:\d{2}:\d{2})?$", s):
        dt = pd.to_datetime(s, dayfirst=True, errors="coerce")
        if not pd.isna(dt): return dt

    # Genel temizlik
    s_clean = re.sub(r"[^\d\.\-:\s]", "", s)
    dt = pd.to_datetime(s_clean, dayfirst=True, errors="coerce")
    if not pd.isna(dt): return dt

    # Yalnız yıl
    yr = pd.to_numeric(s, errors="coerce")
    if not pd.isna(yr):
        try:
            return datetime(int(yr), 1, 1)
        except:
            return pd.NaT

    return pd.NaT

# ========= Özelleştirme tarihi (post) için yardımcılar =========
PRIV_CUTOFF_DT = datetime(2009, 1, 28)          # post==True eşiği
PRE2008_CUTOFF = datetime(2008, 1, 28)          # istersen kullanırsın (yük>80 için farklı eşik gerekiyorsa)

# Yalnızca bu tip kolonlar "özelleştirme" sayılır:
PRIV_DATE_KEYWORDS = [
    ("ozellestirme","tarihi"),
    ("ozellestirme",),
    ("devralma","tarihi"),
    ("kesin","kabul","tarihi"),
    ("isletmeye","alis","tarihi"),
    ("tesis","devreye","tarihi"),
]

# Bunlar hariç tutulur:
EXCLUDE_DATE_PATTERNS = [
    "baslang", "bitis", "bildirim", "ariza", "saat", "sure", "dakika",
    "kesinti", "olay", "start", "end"
]

def pick_priv_date_columns(frame: pd.DataFrame):
    cols = []
    norm_cols = [(c, normalize(c)) for c in frame.columns]
    for keys in PRIV_DATE_KEYWORDS:
        toks = [normalize(k) for k in keys]
        for c, nc in norm_cols:
            if all(t in nc for t in toks):
                cols.append(c)
    return cols

def get_priv_date(sub: pd.DataFrame):
    """Özelleştirme/devralma/kesin kabul/işletmeye alış/tesis devreye kolonlarından (EXCLUDE hariç)
       tarihleri topla ve en erkenini döndür. Yoksa None."""
    cols = [c for c in pick_priv_date_columns(sub)
            if not any(b in normalize(c) for b in EXCLUDE_DATE_PATTERNS)]
    if not cols:
        return None

    dts = []
    for c in cols:
        dts.extend(sub[c].apply(parse_date_or_year).dropna().tolist())
    if not dts:
        return None

    ser = pd.to_datetime(pd.Series(dts).dt.normalize(), errors="coerce").dropna()
    if ser.empty:
        return None
    return ser.min()

def post_cutoff_by_dates(sub):
    """True => 28.01.2009 ve sonrası, False => öncesi, None => tarih yok.
       (Sadece 'özelleştirme' sınıfındaki kolonlara bakar, EXCLUDE edilenleri dışlar.)"""
    dt_min = get_priv_date(sub)
    if dt_min is None:
        return None
    return bool(dt_min >= PRIV_CUTOFF_DT)

# ========= Tip tespiti =========
def describe_type(sub):
    vals = []
    for col in [COL_TRAF_FIDER, COL_TIP_DETAY]:
        if col and col in sub.columns:
            vals += sub[col].dropna().astype(str).tolist()
    n = normalize(" ".join(vals))
    if 'fider' in n: return "FİDER"
    if 'traf' in n: return "TRAFO"
    colnames = " ".join(normalize(c) for c in df.columns)
    if 'fider' in colnames: return "FİDER"
    if 'traf' in colnames: return "TRAFO"
    code_hint = str(sub[CODE_COL].iloc[0]).upper() if not sub.empty else ""
    if re.search(r"(TR|TF|TRAFO)", code_hint): return "TRAFO"
    return "TRAFO"

# ========= Yatırım planı var mı? =========
def detect_yatirim_gorus_column():
    candidates = []
    for c in df.columns:
        nc = normalize(c)
        if any(k in nc for k in ["yatirim","gorus"]) and not any(k in nc for k in ["sonuc","karar","decision","result"]):
            candidates.append(c)
    def score(series):
        s = series.astype(str).str.upper()
        return s.str.contains(r"\bVAR\b|\bYOK\b").mean()
    best = None; best_score = -1
    for c in candidates:
        sc = score(df[c])
        if sc > best_score: best, best_score = c, sc
    return best

COL_YATIRIM_GOR = detect_yatirim_gorus_column()

def yatirim_plani_var_mi(sub):
    if not COL_YATIRIM_GOR or COL_YATIRIM_GOR not in sub.columns:
        return None
    vals = sub[COL_YATIRIM_GOR].astype(str).str.upper().replace({"TRUE":"VAR","FALSE":"YOK","1":"VAR","0":"YOK"})
    if vals.str.contains(r"\bVAR\b").any(): return True
    if vals.str.contains(r"\bYOK\b").any(): return False
    return None

# ===================================== KARAR AĞACI =====================================
# --- İmal yılı kolonlarını bul (yardımcı) ---
IMAL_DATE_KEYWORDS = [
    ("imal","yili"), ("imal","yil"), ("imal",), ("uretim","yili")
]
def pick_imal_date_columns(frame: pd.DataFrame):
    cols = []
    norm_cols = [(c, normalize(c)) for c in frame.columns]
    for keys in IMAL_DATE_KEYWORDS:
        toks = [normalize(k) for k in keys]
        for c, nc in norm_cols:
            if all(t in nc for t in toks):
                cols.append(c)
    return cols

def get_imal_date(sub: pd.DataFrame):
    cols = pick_imal_date_columns(sub)
    if not cols: return None
    dts = []
    for c in cols:
        if c in sub.columns:
            dts.extend(sub[c].apply(parse_date_or_year).dropna().tolist())
    if not dts: return None
    ser = pd.to_datetime(pd.Series(dts).dt.normalize(), errors="coerce").dropna()
    if ser.empty: return None
    try:
        m = ser.mode()
        return m.iloc[0] if not m.empty else ser.median()
    except Exception:
        return ser.max()

# --- (opsiyonel) gri kutu: bakım projesi ile çözülebilir mi?
def bakim_projesiyle_cozulebilir_mi(sub) -> bool:
    if COL_NEDEN and COL_NEDEN in sub.columns:
        txt = " ".join(sub[COL_NEDEN].dropna().astype(str).str.lower().tolist())
        for p in ["bakım projesi","bakim projesi","iletken","sehim","ağaç","agac","dış etken","dis etken","sigorta","şalter","salter"]:
            if p in txt: return True
    return False


# ============================== KARAR AĞACI (7 NET YOL) ==============================

def karar_agaci(sub: pd.DataFrame, code_str: str) -> str:
    """
    7 temel karar yolu:
      1) Yük > 80  &  (özelleştirme/imal) < 28.01.2009               -> YATIRIM
      2) Yük > 80  &  (özelleştirme/imal) >= 28.01.2009              -> OPERASYON
      3) Yük ≤ 80 &  TRAFO & (özelleştirme/imal) < 28.01.2009 & YP=VAR  -> YATIRIM
      4) Yük ≤ 80 &  TRAFO & (özelleştirme/imal) < 28.01.2009 & YP=YOK  -> BAKIM
      5) Yük ≤ 80 &  TRAFO & (özelleştirme/imal) >= 28.01.2009          -> OPERASYON
      6) Yük ≤ 80 &  FİDER & YP=VAR                                     -> YATIRIM
      7) Yük ≤ 80 &  FİDER & YP=YOK                                     -> BAKIM
    """

    # --- ⿡ Yük ortalamasını bul ---
    avg_load = None
    if COL_YUK and COL_YUK in sub.columns:
        ser = sub[COL_YUK].apply(to_float)
        if ser.notna().any():
            avg_load = ser.mean()

    # --- ⿢ Tip (TRAFO/FİDER), tarih, yatırım planı ---
    tip       = describe_type(sub)                 # "TRAFO" / "FİDER" / belirsiz
    post      = post_cutoff_by_dates(sub)          # True: >= 28.01.2009, False: < 28.01.2009, None: yok
    imal_dt   = get_imal_date(sub)                 # None olabilir
    imal_post = None if imal_dt is None else bool(imal_dt >= PRIV_CUTOFF_DT)
    eff_post  = post if post is not None else imal_post  # özell. yoksa imal yılına düş
    yp        = yatirim_plani_var_mi(sub)          # True / False / None

    # --- ⿣ Karar yolları ---
    # [1] ve [2] Yük > 80
    if avg_load is not None and avg_load > 80:
        if eff_post is False:   # < 28.01.2009
            return "YATIRIM"
        if eff_post is True:    # ≥ 28.01.2009
            return "OPERASYON"
        return "YATIRIM"        # tarih bilinmiyor → temkinli yatırım

    # [3]–[7] Yük ≤ 80 (veya yük bilinmiyor)
    if tip == "TRAFO":
        if eff_post is False:
            if yp is True:  return "YATIRIM"  # [3]
            if yp is False: return "BAKIM"    # [4]
            return "BAKIM"
        if eff_post is True:
            return "OPERASYON"                # [5]
        return "BAKIM"

    if tip == "FİDER":
        if yp is True:  return "YATIRIM"      # [6]
        if yp is False: return "BAKIM"        # [7]
        return "BAKIM"

    # Tip belirsiz: trafoya benzer davran, temkinli
    if eff_post is True:  return "OPERASYON"
    if eff_post is False: return "YATIRIM" if yp else "BAKIM"
    return "BAKIM"



# ========= ÖZET =========
def summarize_one_code(code_str):
    code_str = str(code_str).strip().upper()
    sub = df[df[CODE_COL].astype(str).str.upper() == code_str]
    if sub.empty:
        return {"CBS Kodu": code_str, "Karar": "Bulunamadı"}
    tip = describe_type(sub)
    kesinti_sayisi = len(sub)
    total_hours = sub[COL_SURE].apply(to_float).sum() if COL_SURE in sub.columns else np.nan
    avg_load = np.nan
    if COL_YUK in sub.columns:
        al = sub[COL_YUK].apply(to_float)
        if al.notna().any(): avg_load = al.mean()
    karar = karar_agaci(sub, code_str)
    neden_map = {}
    if COL_NEDEN in sub.columns:
        counts = sub[COL_NEDEN].dropna().astype(str).value_counts()
        neden_map = {k:int(v) for k,v in counts.items()}
    return {"CBS Kodu": code_str, "Şebeke Unsuru Tipi": tip, "Kesinti Sayısı": int(kesinti_sayisi),
            "Toplam Süre": float(total_hours) if not pd.isna(total_hours) else np.nan,
            "Ortalama Yük": float(avg_load) if not pd.isna(avg_load) else np.nan,
            "Karar": karar, "Neden Frekansları": neden_map}

def print_pretty(summary):
    print(f"\n--- {summary['CBS Kodu']} ---")
    print(f"Tip: {summary.get('Şebeke Unsuru Tipi')}")
    print(f"Kesinti: {summary.get('Kesinti Sayısı')}")
    print(f"Ortalama Yük: {summary.get('Ortalama Yük')}")
    print(f"Karar: {summary.get('Karar')}")
    nf = summary.get("Neden Frekansları")
    if nf:
        print("Nedenler:")
        for k,v in nf.items(): print(f"  • {k}: {v}")

# ========= ÇALIŞTIRMA =========

if __name__ == "__main__":
    codes = [c.strip() for c in input("CBS kod(lar)ını gir: ").split(",") if c.strip()]
    results = [summarize_one_code(c) for c in codes]
    for r in results: print_pretty(r)
    out = "/content/cbs_batch_summary.xlsx"
    pd.DataFrame(results).to_excel(out, index=False)
    print(f"\n📦 Sonuç kaydedildi → {out}")


CBS kod(lar)ını gir:  TAT551TR1P1C



--- TAT551TR1P1C ---
Tip: FİDER
Kesinti: 19
Ortalama Yük: 11.038000000000002
Karar: YATIRIM
Nedenler:
  • Olumsuz Hava Sartlari - Sigorta Arıza: 7
  • Bakim Ihtiyaci - Sigorta Arıza: 6
  • Agac ve Dis Etkenler - Sigorta Arıza: 3
  • -Ekonomik Ömür - Sigorta Arıza: 2
  • Asiri Yuk - Sigorta Arıza: 1

📦 Sonuç kaydedildi → /content/cbs_batch_summary.xlsx


**ARAYÜZ KODU**

In [5]:
!pip install gradio pandas numpy openpyxl plotly --quiet

import gradio as gr
import pandas as pd
import numpy as np
import re
from datetime import datetime
import plotly.express as px

# ================== GENEL YARDIMCI FONKSİYONLAR ==================
TR_MAP = str.maketrans({
    "İ": "i", "I": "i", "ı": "i",
    "Ş": "s", "ş": "s",
    "Ğ": "g", "ğ": "g",
    "Ü": "u", "ü": "u",
    "Ö": "o", "ö": "o",
    "Ç": "c", "ç": "c",
})

def norm_header(s: str) -> str:
    s = str(s)
    s = s.translate(TR_MAP).lower()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return s.strip()

def clean_str(x):
    """
    Görünen metinler için: Türkçe karakterleri BOZMADAN
    sadece boşluk/tire temizliği ve basit formatlama yap.
    """
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).replace("\n", " ").replace("\r", " ").strip()

    # Birden fazla boşluk -> tek boşluk
    s = re.sub(r"\s+", " ", s)
    s = s.replace("–", "-").replace("—", "-")
    s = s.replace(" - ", "-").replace("- ", "-").replace(" -", "-")

    # İlk harf büyük, kalan küçük (özellikle şehir / ilçe için)
    if len(s) > 0:
        s = s[0].upper() + s[1:].lower()

    return s.strip()

def to_float(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    s = str(x).replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    if s == "":
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

def parse_date(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    try:
        return pd.to_datetime(x, dayfirst=True, errors="coerce")
    except Exception:
        return None

def find_column(df, token_sets, fallback=None):
    headers = list(df.columns)
    norms = [norm_header(h) for h in headers]

    for tokens in token_sets:
        toks = [norm_header(t) for t in tokens]
        for col, nh in zip(headers, norms):
            if all(t in nh for t in toks):
                return col

    if fallback is not None:
        fb = norm_header(fallback)
        for col, nh in zip(headers, norms):
            if fb == nh:
                return col
    return None

def load_to_percent(val):
    """
    Excel'deki yüzdelik yük:
    - 0 < x < 2  ise: 0.85 -> 85.0 (%)
    - 2 <= x <= 100 ise: olduğu gibi (%)
    """
    v = to_float(val)
    if np.isnan(v):
        return np.nan
    if 0 < v < 2:
        v = v * 100.0
    return v

# ================== TÜRKİYE İL → KOORD SÖZLÜĞÜ ==================
# (Tüm iller duruyor, ama senin veri şu an Kırıkkale & Zonguldak bölgesinden.)
TURKEY_CITIES = {
    "adana": (37.0, 35.3),
    "adiyaman": (37.8, 38.3),
    "afyonkarahisar": (38.8, 30.5),
    "agri": (39.7, 43.0),
    "amasya": (40.6, 35.8),
    "ankara": (39.9, 32.9),
    "antalya": (36.9, 30.7),
    "artvin": (41.2, 41.8),
    "aydin": (37.8, 27.8),
    "balikesir": (39.6, 27.9),
    "bilecik": (40.1, 29.9),
    "bingol": (38.9, 40.5),
    "bitlis": (38.4, 42.1),
    "bolu": (40.7, 31.6),
    "burdur": (37.7, 30.3),
    "bursa": (40.2, 29.0),
    "canakkale": (40.1, 26.4),
    "cankiri": (40.6, 33.6),
    "corum": (40.5, 34.9),
    "denizli": (37.8, 29.1),
    "diyarbakir": (37.9, 40.2),
    "edirne": (41.7, 26.6),
    "elazig": (38.7, 39.2),
    "erzincan": (39.7, 39.5),
    "erzurum": (39.9, 41.3),
    "eskisehir": (39.8, 30.5),
    "gaziantep": (37.1, 37.4),
    "giresun": (40.9, 38.4),
    "gumushane": (40.5, 39.5),
    "hakkari": (37.6, 43.7),
    "hatay": (36.2, 36.1),
    "isparta": (37.8, 30.5),
    "mersin": (36.8, 34.6),
    "istanbul": (41.0, 28.9),
    "izmir": (38.4, 27.1),
    "kars": (40.6, 43.1),
    "kastamonu": (41.4, 33.8),
    "kayseri": (38.7, 35.5),
    "kirklareli": (41.7, 27.2),
    "kirsehir": (39.1, 34.2),
    "kocaeli": (40.8, 29.9),
    "konya": (37.9, 32.5),
    "kutahya": (39.4, 29.9),
    "malatya": (38.3, 38.3),
    "manisa": (38.6, 27.4),
    "kahramanmaras": (37.6, 36.9),
    "mardin": (37.3, 40.7),
    "mugla": (37.2, 28.4),
    "mus": (38.7, 41.5),
    "nevsehir": (38.6, 34.7),
    "nigde": (37.9, 34.7),
    "ordu": (40.9, 37.9),
    "rize": (41.0, 40.5),
    "sakarya": (40.8, 30.4),
    "samsun": (41.3, 36.3),
    "siirt": (37.9, 41.9),
    "sinop": (42.0, 35.1),
    "sivas": (39.7, 37.0),
    "tekirdag": (40.9, 27.5),
    "tokat": (40.3, 36.6),
    "trabzon": (41.0, 39.7),
    "tunceli": (39.1, 39.5),
    "sanliurfa": (37.2, 38.8),
    "usak": (38.7, 29.4),
    "van": (38.5, 43.4),
    "yozgat": (39.8, 34.8),
    "zonguldak": (41.4535, 31.7890),
    "aksaray": (38.4, 34.0),
    "bayburt": (40.3, 40.2),
    "karaman": (37.2, 33.2),
    "kirikkale": (39.8468, 33.5153),
    "batman": (37.9, 41.1),
    "sirnak": (37.5, 42.5),
    "bartin": (41.6, 32.3),
    "ardahan": (41.1, 42.7),
    "igdir": (39.9, 44.0),
    "yalova": (40.6, 29.3),
    "karabuk": (41.2, 32.6),
    "kilis": (36.7, 37.1),
    "osmaniye": (37.1, 36.2),
    "duzce": (40.8, 31.2),
}

# ================== KARAR AĞACI (7 YOL) ==================
def karar_agaci(avg_load, tip, tarih, yatirim_plani_var):
    cutoff = datetime(2009, 1, 28)
    eff_post = None
    if tarih is not None:
        eff_post = tarih >= cutoff

    if avg_load is not None and avg_load > 80:
        if eff_post is False:
            return "YATIRIM"
        if eff_post is True:
            return "OPERASYON"
        return "YATIRIM"

    if tip == "TRAFO":
        if eff_post is False:
            if yatirim_plani_var is True:
                return "YATIRIM"
            if yatirim_plani_var is False:
                return "BAKIM"
            return "BAKIM"
        if eff_post is True:
            return "OPERASYON"
        return "BAKIM"

    if tip == "FİDER":
        if yatirim_plani_var is True:
            return "YATIRIM"
        if yatirim_plani_var is False:
            return "BAKIM"
        return "BAKIM"

    if eff_post is True:
        return "OPERASYON"
    if eff_post is False:
        return "YATIRIM" if yatirim_plani_var else "BAKIM"
    return "BAKIM"

def karar_badge_html(karar: str) -> str:
    if karar == "YATIRIM":
        bg = "linear-gradient(135deg,#2dd4bf,#22c1c3)"
    elif karar == "OPERASYON":
        bg = "linear-gradient(135deg,#60a5fa,#3b82f6)"
    else:
        bg = "linear-gradient(135deg,#facc15,#f97316)"

    return f"""
<div style="
    display:inline-block;
    padding:10px 18px;
    border-radius:999px;
    background:{bg};
    color:#020617;
    font-weight:700;
    box-shadow:0 0 18px rgba(120,200,255,0.55);
">
    Karar: {karar}
</div>
"""

# ================== GLOBAL GEÇMİŞ ==================
HISTORY_CODES = []

# ================== ANA ANALİZ ==================
def analyze_file(file, cbs_code):
    if file is None:
        return ("Lütfen Excel dosyasını yükleyin.", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)

    try:
        df = pd.read_excel(file.name)
    except Exception as e:
        return (f"Dosya okunamadı: {e}", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)

    # kolon bulma
    COL_CBS   = find_column(df, [["cbs","kodu"],["cbs","kod"],["cbs"]])
    COL_IL    = find_column(df, [["il","3a"],["il "]])
    COL_ILCE  = find_column(df, [["ilce","3b"],["ilce"]])
    COL_NEDEN = find_column(df, [["kesinti","nedenine","iliskin","aciklama"],["kesinti","nedeni"]])
    COL_BAS   = find_column(df, [["kesinti","baslama","tarihi"],["baslama","tarih"]])
    COL_BIT   = find_column(df, [["kesinti","sona","erme","tarihi"],["bitis","tarih"]])
    COL_SURE  = find_column(df, [["kesinti","suresi","saat"],["kesinti","suresi"]])
    COL_YUK   = find_column(df, [["yuzdelik","yuk","profil"],["yuk","profil"],["yuk","%"]])
    COL_TIP   = find_column(df, [["trafo","fider"],["trafo"],["fider"]])
    COL_YP    = find_column(df, [["yatirim","gorusu"],["yatirim","görüşü"]])
    COL_IMAL  = find_column(df, [["imal","yili"],["uretim","yili"]])

    if COL_CBS is None:
        return ("Excel'de CBS kodu kolonu bulunamadı (örn. 'CBS KODU').", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)

    code = clean_str(cbs_code).upper()
    if not code:
        return ("Lütfen CBS kodu girin.", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)

    sub = df[df[COL_CBS].astype(str).str.upper().str.strip() == code]

    if sub.empty:
        if code not in HISTORY_CODES:
            HISTORY_CODES.append(code)
        hist_md = "### 🔁 Geçmiş Aramalar\n" + ", ".join(HISTORY_CODES)
        return (f"{code}** için kayıt bulunamadı.", "", "", pd.DataFrame(),
                "", None, None, None, hist_md, HISTORY_CODES)

    # metrikler
    kesinti_sayisi = len(sub)

    total_hours_ser = sub[COL_SURE].apply(to_float) if COL_SURE else pd.Series([np.nan])
    total_hours = total_hours_ser.sum() if total_hours_ser.notna().any() else 0

    # Yük: 0–2 arası ise *100, 2–100 arası ise direkt
    if COL_YUK:
        loads = sub[COL_YUK].apply(load_to_percent)
        avg_load = loads.mean() if loads.notna().any() else None
    else:
        loads = pd.Series([np.nan])
        avg_load = None

    tip_raw = clean_str(sub[COL_TIP].iloc[0]) if COL_TIP else ""
    tip_low = tip_raw.lower()
    tip = "TRAFO" if "traf" in tip_low else "FİDER"

    dts = sub[COL_BAS].apply(parse_date).dropna() if COL_BAS else pd.Series([], dtype="datetime64[ns]")
    first_dt = dts.min() if not dts.empty else None

    imal_dt = None
    if COL_IMAL and COL_IMAL in sub.columns:
        raw_imal = sub[COL_IMAL].iloc[0]
        if not pd.isna(raw_imal):
            try:
                y = int(str(raw_imal)[:4])
                imal_dt = datetime(y, 1, 1)
            except:
                imal_dt = None

    eff_date = first_dt or imal_dt

    yp_val = None
    if COL_YP and COL_YP in sub.columns:
        vals = sub[COL_YP].astype(str).str.upper()
        if (vals.str.contains("VAR")).any():
            yp_val = True
        elif (vals.str.contains("YOK")).any():
            yp_val = False

    karar = karar_agaci(avg_load, tip, eff_date, yp_val)

    il   = clean_str(sub[COL_IL].iloc[0])   if COL_IL   else "-"
    ilce = clean_str(sub[COL_ILCE].iloc[0]) if COL_ILCE else "-"

    # neden frekansları
    nf_dict = {}
    if COL_NEDEN:
        neden_ser = sub[COL_NEDEN]
        for v in neden_ser:
            if pd.isna(v):
                continue
            clean_v = clean_str(v)
            if clean_v == "":
                continue
            nf_dict[clean_v] = nf_dict.get(clean_v, 0) + 1

    # kayıtlar
    kayitlar = []
    for _, row in sub.iterrows():
        rec = {
            "Başlangıç": parse_date(row[COL_BAS]) if COL_BAS else None,
            "Bitiş": parse_date(row[COL_BIT]) if COL_BIT else None,
            "Süre (saat)": to_float(row[COL_SURE]) if COL_SURE else np.nan,
            "Neden": clean_str(row[COL_NEDEN]) if COL_NEDEN else "",
            "Yük (%)": load_to_percent(row[COL_YUK]) if COL_YUK else np.nan,
            "İl": clean_str(row[COL_IL]) if COL_IL else "",
            "İlçe": clean_str(row[COL_ILCE]) if COL_ILCE else "",
        }
        kayitlar.append(rec)
    detay_df = pd.DataFrame(kayitlar)

    # karar kartı
    badge_html = karar_badge_html(karar)
    card_md = f"""
### 📦 CBS Kodu: *{code}*

{badge_html}

<br/>

- Şebeke Unsuru: *{tip}*
- Kesinti Sayısı: *{kesinti_sayisi}*
- Toplam Süre (saat): *{round(total_hours,2)}*
- Ortalama Yük (%): *{round(avg_load,1) if avg_load is not None else "-"}*
"""

    # ek bilgiler + özet
    if eff_date is not None:
        year_min = eff_date.year
        year_max = (sub[COL_BAS].apply(parse_date).dropna().max().year
                    if COL_BAS and sub[COL_BAS].notna().any() else year_min)
        year_span = f"{year_min}-{year_max}" if year_min != year_max else str(year_min)
    else:
        year_span = "-"

    summary_sentence = (
        f"Bu CBS kodunda {year_span} döneminde toplam {kesinti_sayisi} kesinti kaydı bulunmuştur. "
        f"Toplam kesinti süresi yaklaşık {round(total_hours,2)} saattir ve "
        f"ortalama yük yaklaşık %{round(avg_load,1) if avg_load is not None else 0} seviyesindedir."
    )

    ek_md = f"""
### 🗂 Ek Bilgiler

- İl: *{il}*
- İlçe: *{ilce}*
- Kayıtlı kesinti sayısı: *{kesinti_sayisi}*
- Dönem: *{year_span}*

*Özet:* {summary_sentence}
"""

    # neden frekansları md
    if nf_dict:
        nf_lines = "\n".join([f"- *{k}*: {v} kez" for k, v in nf_dict.items()])
        nf_md = "### ⚡ Kesinti Nedenleri (Frekans)\n\n" + nf_lines
    else:
        nf_md = "### ⚡ Kesinti Nedenleri\n\nKayıtlı neden bulunamadı."

    explanation_md = ""

    # grafikler
    fig_years = None
    if not detay_df.empty and detay_df["Başlangıç"].notna().any():
        years = pd.to_datetime(detay_df["Başlangıç"]).dt.year.dropna()
        year_counts = years.value_counts().sort_index()
        fig_years = px.bar(
            x=year_counts.index.astype(str),
            y=year_counts.values,
            labels={"x": "Yıl", "y": "Kesinti Sayısı"},
            title="Yıllara Göre Kesinti Sayısı"
        )
        fig_years.update_layout(
            template="plotly_dark",
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            font_color="#e5edff"
        )

    fig_reasons = None
    if nf_dict:
        r_df = pd.DataFrame({"Neden": list(nf_dict.keys()), "Sayı": list(nf_dict.values())})
        fig_reasons = px.bar(
            r_df, x="Neden", y="Sayı",
            title="Kesinti Nedenlerinin Dağılımı"
        )
        fig_reasons.update_layout(
            template="plotly_dark",
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            font_color="#e5edff"
        )

    # Kesinti Süresi Dağılımı GRAFİĞİ TAMAMEN KALDIRILDI

    fig_location = None
    if il != "-" and il:
        key = norm_header(il)
        if key in TURKEY_CITIES:
            lat, lon = TURKEY_CITIES[key]
            loc_df = pd.DataFrame({"lat": [lat], "lon": [lon], "İl": [il], "İlçe": [ilce]})
            fig_location = px.scatter_geo(
                loc_df,
                lat="lat", lon="lon",
                hover_name="İlçe",
                text="İl",
                scope="asia",
                title="Lokasyon (Türkiye Üzerinde Yaklaşık Konum)"
            )
            fig_location.update_geos(
                center={"lat": 39.0, "lon": 35.0},
                lataxis_range=[35, 43],
                lonaxis_range=[25, 45],
                showland=True,
                showcountries=True,
            )
            fig_location.update_layout(
                template="plotly_dark",
                paper_bgcolor="rgba(0,0,0,1)",
                plot_bgcolor="rgba(0,0,0,0)",
                font_color="#e5edff"
            )
        else:
            fig_location = px.scatter(
                x=[0], y=[0], text=[f"{il} / {ilce}"],
                title="Lokasyon Gösterimi",
            )
            fig_location.update_traces(textposition="top center")
            fig_location.update_xaxes(visible=False)
            fig_location.update_yaxes(visible=False)
            fig_location.update_layout(
                template="plotly_dark",
                paper_bgcolor="rgba(0,0,0,0)",
                plot_bgcolor="rgba(0,0,0,0)",
                font_color="#e5edff"
            )

    # geçmiş
    if code not in HISTORY_CODES:
        HISTORY_CODES.append(code)
    hist_md = "### 🔁 Geçmiş Aramalar\n" + ", ".join(HISTORY_CODES)

    return (card_md, ek_md, nf_md, detay_df,
            explanation_md, fig_years, fig_reasons, fig_location,
            hist_md, HISTORY_CODES)

# ================== KARAR AÇIKLAMASI ==================
def explain_decision(file, cbs_code):
    if file is None:
        return "Önce Excel dosyasını yükleyin."
    card_md, *_ = analyze_file(file, cbs_code)

    code = clean_str(cbs_code).upper()
    lines = [f"### 📘 {code} için Karar Açıklaması\n"]
    if "Karar: YATIRIM" in card_md:
        lines.append("- Ortalama yük yüksek veya eski tarihli bir unsur olduğu için yatırım önceliği öne çıkıyor.")
        lines.append("- 28.01.2009 öncesi dönemde olması ve kesinti profili yatırım ihtiyacını destekliyor.")
    elif "Karar: OPERASYON" in card_md:
        lines.append("- Şebeke unsuru görece yeni (2009 sonrası) kabul ediliyor.")
        lines.append("- Yük ve kesinti davranışı, operasyonel müdahale ile yönetilebilir seviyede.")
        lines.append("- Bu nedenle operasyonel aksiyonlar önceliklidir.")
    else:  # BAKIM
        lines.append("- Yük seviyesi kritik eşiklerin altında.")
        lines.append("- Kesinti profili, periyodik ve hedefli bakım ile yönetilebilir durumda.")
        lines.append("- Bu nedenle sonuç *BAKIM* yönünde; yatırım önceliği daha düşüktür.")
    return "\n".join(lines)

# ================== GEÇMİŞ KODDAN GÖSTER ==================
def show_from_history(file, history_code):
    if file is None:
        return ("Lütfen Excel dosyasını yükleyin.", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)
    if not history_code:
        return ("Lütfen geçmiş listeden bir CBS kodu seçin.", "", "", pd.DataFrame(),
                "", None, None, None, "Henüz kayıt yok.", HISTORY_CODES)
    return analyze_file(file, history_code)

# ================== NEON MAVİ GLASSMORPHISM TEMA ==================
CUSTOM_CSS = """
/* Background Gradient + Glow */
body {
    background: radial-gradient(circle at 20% 20%, #4da3ff33 0%, #0a0f1e 60%) !important;
    font-family: 'Inter', 'Segoe UI', sans-serif !important;
    color: #e8ecf2 !important;
}

/* Glassmorphism Panels */
.gradio-container, .gr-box, .gr-panel {
    background: rgba(255, 255, 255, 0.04) !important;
    backdrop-filter: blur(18px) !important;
    border: 1px solid rgba(255, 255, 255, 0.09) !important;
    border-radius: 22px !important;
    box-shadow: 0 0 35px rgba(80, 150, 255, 0.25) !important;
    padding: 18px !important;
}

/* Header */
h1 {
    font-size: 42px !important;
    font-weight: 800 !important;
    text-align: center;
    color: #d3e9ff !important;
    letter-spacing: -0.5px;
    margin-bottom: -5px !important;
}
h2, h3 {
    color: #c7d8f5 !important;
}

/* Inputs */
input, textarea {
    background: rgba(255,255,255,0.08) !important;
    border-radius: 14px !important;
    border: none !important;
    padding: 12px 14px !important;
    color: #e7ecf4 !important;
    font-weight: 500 !important;
}

/* Buttons (capsule) */
button {
    background: linear-gradient(135deg, #66b3ff, #3d8bfd) !important;
    border-radius: 18px !important;
    color: white !important;
    border: none !important;
    font-weight: 700 !important;
    padding: 12px 18px !important;
    font-size: 16px !important;
    transition: 0.15s ease-in-out !important;
}
button:hover {
    transform: scale(1.05);
    box-shadow: 0px 0px 18px rgba(120,180,255,0.55);
}

/* Dropdowns */
select {
    background: rgba(255,255,255,0.08) !important;
    color: #e5edff !important;
    border-radius: 14px !important;
    padding: 10px !important;
}

/* Dataframe */
.gr-dataframe table {
    background: rgba(255,255,255,0.05) !important;
    color: #cfe5ff !important;
}

/* Plots */
.gr-plot {
    background: rgba(255,255,255,0.06) !important;
    border-radius: 20px !important;
    padding: 10px !important;
}
"""

# ================== GRADIO ARAYÜZÜ ==================
with gr.Blocks(css=CUSTOM_CSS) as demo:
    gr.Markdown("""
    <div style='text-align:center; margin-top:20px; margin-bottom:10px'>
        <h1>Walter Watt X Shenergy<br>Yatırım Bakım Sorgulama Sistemi</h1>
        <p style='color:#9fb3ff; font-size:15px; margin-top:4px;'>
            CBS kodunu gir, sistem karar ağacı ile otomatik yatırım / bakım / operasyon analizini oluştursun.
        </p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Excel Yükle", file_types=[".xlsx"])
        with gr.Column(scale=1):
            cbs_input = gr.Textbox(label="CBS Kodu", placeholder="Örn: TAM175TR1P1D")

    with gr.Row():
        analyze_btn = gr.Button("Analiz Et")
        explain_btn = gr.Button("Karar Açıklaması")
        history_btn = gr.Button("Geçmiş Kararı Göster")

    karar_md = gr.Markdown()
    ek_md = gr.Markdown()
    nf_md = gr.Markdown()

    detay_tbl = gr.Dataframe(
        label="Kesinti Kayıtları",
        headers=["Başlangıç","Bitiş","Süre (saat)","Neden","Yük (%)","İl","İlçe"],
        row_count=0, col_count=7
    )

    explanation_md = gr.Markdown()

    with gr.Row():
        plot_years = gr.Plot(label="Yıllara Göre Kesinti Sayısı")
        plot_reasons = gr.Plot(label="Kesinti Nedenlerinin Dağılımı")

    with gr.Row():
        plot_location = gr.Plot(label="Lokasyon Gösterimi")

    history_md = gr.Markdown()
    history_dd = gr.Dropdown(label="Geçmiş CBS Kodları", choices=[], interactive=True)

    # ANALİZ
    analyze_btn.click(
        fn=analyze_file,
        inputs=[file_input, cbs_input],
        outputs=[
            karar_md, ek_md, nf_md,
            detay_tbl,
            explanation_md,
            plot_years, plot_reasons, plot_location,
            history_md, history_dd
        ]
    )

    # KARAR AÇIKLAMASI
    explain_btn.click(
        fn=explain_decision,
        inputs=[file_input, cbs_input],
        outputs=explanation_md
    )

    # GEÇMİŞTEN YÜKLE
    history_btn.click(
        fn=show_from_history,
        inputs=[file_input, history_dd],
        outputs=[
            karar_md, ek_md, nf_md,
            detay_tbl,
            explanation_md,
            plot_years, plot_reasons, plot_location,
            history_md, history_dd
        ]
    )

demo.launch(share=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://27a2cc42bb0d36ac44.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
